# Two Body Indirect Optimal Control

In [1]:
from sympy import *
init_printing()

## Dynamics


In [21]:
# state
x, y, z, vx, vy, vz, m = symbols('x y z v_x v_y v_z m')
r = Matrix([x, y, z])
v = Matrix([vx, vy, vz])
s = Matrix([r, v, [m]])
rmag = sqrt(r[0]**2 + r[1]**2 + r[2]**2)

# costate
lx, ly, lz, lvx, lvy, lvz, lm = symbols('\lambda_x \lambda_y \lambda_z \lambda_{v_x} \lambda_{v_y} \lambda_{v_z} \lambda_{m}')
lr = Matrix([lx, ly, lz])
lv = Matrix([lvx, lvy, lvz])
l = Matrix([lr, lv, [lm]])

# full state
fs = Matrix([s, l])

# controls
umag, ux, uy, uz = symbols('u u_x u_y u_z')
u = umag*Matrix([ax, ay, az])

# parametres 
#(c1 = Tmax, c2 = Tmax/Isp/g0, eps = homotopy (MO=0, PO=1))
c1, c2, mu, eps = symbols('c_1 c_2 \mu \epsilon')

# dynamics
ds = Matrix([
    v, 
    u*c1/m - mu*r/rmag**3, 
    [-umag*c2]
])
ds

⎡              vₓ              ⎤
⎢                              ⎥
⎢             v_y              ⎥
⎢                              ⎥
⎢             v_z              ⎥
⎢                              ⎥
⎢        \mu⋅x         c₁⋅u⋅uₓ ⎥
⎢- ───────────────── + ─────── ⎥
⎢                3/2      m    ⎥
⎢  ⎛ 2    2    2⎞              ⎥
⎢  ⎝x  + y  + z ⎠              ⎥
⎢                              ⎥
⎢        \mu⋅y         c₁⋅u⋅u_y⎥
⎢- ───────────────── + ────────⎥
⎢                3/2      m    ⎥
⎢  ⎛ 2    2    2⎞              ⎥
⎢  ⎝x  + y  + z ⎠              ⎥
⎢                              ⎥
⎢        \mu⋅z         c₁⋅u⋅u_z⎥
⎢- ───────────────── + ────────⎥
⎢                3/2      m    ⎥
⎢  ⎛ 2    2    2⎞              ⎥
⎢  ⎝x  + y  + z ⎠              ⎥
⎢                              ⎥
⎣            -c₂⋅u             ⎦

## Indirect Optimal Control

In [123]:
# homotopic cost Lagrangian
L = c2*(umag - eps*umag*(1 - umag))

# Hamiltonian
H = l.dot(ds) + L

# costate dynamics
dl = -Matrix([H.diff(i) for i in s])
simplify(dl)

⎡     ⎛                ⎛     2    2    2⎞                                     
⎢-\mu⋅⎝- \lambda_{v_x}⋅⎝- 2⋅x  + y  + z ⎠ + 3⋅x⋅(\lambda_{v_y}⋅y + \lambda_{v_
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                               5/2                           
⎢                                 ⎛ 2    2    2⎞                              
⎢                                 ⎝x  + y  + z ⎠                              
⎢                                                                             
⎢      ⎛                ⎛ 2      2    2⎞                                      
⎢ -\mu⋅⎝- \lambda_{v_y}⋅⎝x  - 2⋅y  + z ⎠ + 3⋅y⋅(\lambda_{v_x}⋅x + \lambda_{v_z
⎢ ────────────────────────────────────────────────────────────────────────────
⎢                                               5/2                           
⎢                                 ⎛ 2    2    2⎞                              
⎢                                 ⎝x  + y  + z ⎠    

In [64]:
# differentiate Hamiltionian
Hux = H.diff(ux)
Huy = H.diff(uy)
Huz = H.diff(uz)
Hu = H.diff(umag)
sys = [Hux, Huy, Huz, Hu]

In [135]:
# solve for optimal stearing direction
uxs = solve(sys, ux)[ux]
uys = solve(sys, uy)[uy]
uzs = solve(sys, uz)[uz]
us = Matrix([uxs, uys, uzs])
Hs = H.subs([(ux, uxs), (uy, uys), (uz, uzs)])
simplify(Hs) # the Hamiltonian with optimal stearing

# how to find switching function?

               2                                                 \lambda_{v_x}
- \epsilon⋅c₂⋅u  + \lambdaₓ⋅vₓ + \lambda_y⋅v_y + \lambda_z⋅v_z - ─────────────
                                                                              
                                                                  ⎛ 2    2    
                                                                  ⎝x  + y  + z

⋅\mu⋅x   \lambda_{v_y}⋅\mu⋅y   \lambda_{v_z}⋅\mu⋅z
────── - ─────────────────── - ───────────────────
  3/2                   3/2                   3/2 
2⎞        ⎛ 2    2    2⎞        ⎛ 2    2    2⎞    
 ⎠        ⎝x  + y  + z ⎠        ⎝x  + y  + z ⎠    

In [133]:
simplify(Hs.diff(umag))

-2⋅\epsilon⋅c₂⋅u